In [1]:
import torch

In [25]:
from lavis.models.blip2_models.Qformer import BertLMHeadModel, BertModel, BertOnlyMLMHead

In [26]:
query_tokens = torch.load("/mnt/d/QFormer/query_tokens.pth", map_location="cpu")
image_embeds = torch.load("/mnt/d/QFormer/image_embeds.pth", map_location="cpu")
image_atts = torch.load("/mnt/d/QFormer/image_atts.pth", map_location="cpu")

In [4]:
query_tokens.shape

torch.Size([100, 32, 768])

In [5]:
image_embeds.shape

torch.Size([100, 257, 1408])

In [6]:
image_atts.shape

torch.Size([100, 257])

In [27]:
from transformers.models.bert.configuration_bert import BertConfig
from transformers.configuration_utils import PretrainedConfig

In [28]:
pretrained_config = PretrainedConfig()
pretrained_config = pretrained_config.from_pretrained("/mnt/d/QFormer/config.json")

In [29]:
query_output_real = torch.load("/mnt/d/QFormer/query_output.pth", map_location="cpu")

In [30]:
state_dict = torch.load("/mnt/d/QFormer/self_Qformer_state_dict_init.pth", map_location="cpu")

In [11]:
print("total params: ", len(state_dict))
for name, tensor in state_dict.items():
    print(name, tensor.shape)

total params:  336
bert.embeddings.position_ids torch.Size([1, 512])
bert.embeddings.word_embeddings.weight torch.Size([30523, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.enco

In [32]:
qFormer = BertLMHeadModel(pretrained_config)
print("total params: ", len(list(qFormer.named_parameters())))
for name, tensor in qFormer.named_parameters():
    print(name, tensor.shape)

total params:  334
bert.embeddings.word_embeddings.weight torch.Size([30522, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.crossattention.self.query.weight torch

In [33]:
qFormer.resize_token_embeddings(30523)
print("total params: ", len(list(qFormer.named_parameters())))
for name, tensor in qFormer.named_parameters():
    print(name, tensor.shape)

total params:  333
bert.embeddings.word_embeddings.weight torch.Size([30523, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.crossattention.self.query.weight torch

In [36]:
qFormer.get_input_embeddings().num_embeddings

30523

In [14]:
qFormer.load_state_dict(state_dict)

<All keys matched successfully>

In [15]:
# for name, param in qFormer.named_parameters():
#     if "_query" in name:
#         key_orig = name.replace("_query", "")
#         param.data.copy_(state_dict[key_orig])

In [16]:
for name, param in qFormer.named_parameters():
    if "_query" in name:
        key_orig = name.replace("_query", "")
        param.data = state_dict[key_orig].data
        # param.data.copy_(state_dict[key_orig])

In [51]:
query_tokens.shape

torch.Size([100, 32, 768])

In [18]:
query_output = qFormer.bert(
           query_embeds=query_tokens,
           encoder_hidden_states=image_embeds,
           encoder_attention_mask=image_atts,
           use_cache=True,
           return_dict=True,
           )

In [54]:
hs1 = query_output.last_hidden_state

In [57]:
hs1

tensor([[[-1.9438e-01,  7.3882e-03,  6.3201e-01,  ...,  8.8565e-02,
           8.0319e-02, -1.7347e-01],
         [-1.8025e-01,  1.7617e-02,  5.4297e-01,  ...,  7.7662e-02,
           9.2035e-02, -2.6754e-01],
         [-4.8205e-02, -4.9403e-02,  6.0834e-01,  ..., -6.8633e-02,
           2.9498e-02, -3.5586e-01],
         ...,
         [-1.0602e-01, -1.1701e-01,  6.5464e-01,  ..., -2.2739e-03,
           1.6734e-02, -3.0597e-01],
         [-1.9081e-01, -8.9147e-02,  6.2723e-01,  ..., -2.7298e-02,
           2.5504e-02, -2.7357e-01],
         [-1.6533e-01,  4.0572e-03,  6.0402e-01,  ...,  7.5947e-02,
           2.8548e-02, -3.6183e-01]],

        [[ 9.7987e-02,  3.8709e-01,  4.9894e-01,  ...,  2.6643e-01,
          -7.2123e-02, -6.5843e-02],
         [ 6.0674e-02,  4.1230e-01,  4.0371e-01,  ...,  2.2618e-01,
          -3.8215e-02, -1.7246e-01],
         [ 3.1859e-01,  2.8222e-01,  4.5629e-01,  ...,  9.2004e-05,
          -1.2570e-01, -4.0215e-01],
         ...,
         [ 7.3456e-02,  3

In [ ]:
query_output2 = qFormer.bert(
           query_embeds=query_tokens,
           encoder_hidden_states=image_embeds,
           encoder_attention_mask=image_atts,
           use_cache=True,
           return_dict=True,
           )

In [ ]:
hs2 = query_output2.last_hidden_state

In [ ]:
hs2.equal(hs1)

In [ ]:
hs3 = query_output_real.last_hidden_state

In [ ]:
hs3

In [ ]:
hs3.equal(hs1)

In [ ]:
import torch
l = torch.arange(100).reshape(10, 10)

In [ ]:
l.shape
l.t().shape

In [ ]:
l.t()

In [ ]:
l.T

In [ ]:
import mindspore.ops as P
m = P.arange(100).reshape(10, 10)

In [ ]:
m.T

In [ ]:
dense.weight.t()

In [ ]:
from mindspore import ops as P
expand_x = P.BroadcastTo((1, pretrained_config.max_position_embeddings))

In [ ]:
from mindspore import numpy as np
l = P.arange(pretrained_config.max_position_embeddings)

In [ ]:
l.shape

In [ ]:
import torch

In [ ]:
n = torch.arange(pretrained_config.max_position_embeddings)

In [ ]:
nn = n.expand((1, -1))

In [ ]:
expand_x(l).shape == nn.shape

In [ ]:
from mindspore.common.initializer import initializer, One, Zero

In [ ]:
tensor1 = initializer(Zero(), 100)

In [ ]:
from mindspore import Parameter

In [ ]:
tensor1 = Parameter(tensor1)

In [ ]:
tensor1

### ------------------------------------------------------------ split line ---------------------------------------------------------------

## <\br> Initialize mindspore implement of Q-Former

In [1]:
# from transformers.models.bert.configuration_bert import BertConfig
from transformers.configuration_utils import PretrainedConfig

pretrained_config = PretrainedConfig()
pretrained_config = pretrained_config.from_pretrained("/mnt/d/QFormer/config.json")

In [2]:
import mindspore as ms
ms.set_context(mode=ms.GRAPH_MODE, device_target="CPU")

In [3]:
from mindformers.models.blip2 import QFormer

/mnt/c/Users/GeneYu/desktop/mindformers/mindformers/core/clip_grad.py:39: RuntimeWarning: divide by zero encountered in log
  inf = Tensor(np.log(0.0), mstype.float32)


In [4]:
qFormer2 = QFormer.BertLMHeadModel(pretrained_config)

ValueError: Parameter config in `BertLMHeadModel(config)` should be an instance of class `BaseConfig`. To create a model from a pretrained model use `model = BertLMHeadModel.from_pretrained(PRETRAINED_MODEL_NAME)`

In [ ]:
def mindspore_params(network):
    ms_params = {}
    param_cnt = 0
    for param in network.get_parameters():
        name = param.name
        value = param.data.asnumpy()
        print(name, value.shape)
        ms_params[name] = value
        param_cnt += 1
    print("total params: ", param_cnt)
    return ms_params

qFormer2_params = mindspore_params(qFormer2)

In [6]:
qFormer2.resize_token_embeddings(30523)

bert.embeddings.word_embeddings.embedding_table
embedding_table
bert.embeddings.word_embeddings.embedding_table
before set_input:  bert.embeddings.word_embeddings.embedding_table
after set_input, new :  word_embeddings.embedding_table
after set_input, self.get_input_embeddings() :  word_embeddings.embedding_table
after get_output_embeddings():  word_embeddings.embedding_table
word_embeddings.embedding_table
word_embeddings.embedding_table
word_embeddings.embedding_table


Embedding<vocab_size=30523, embedding_size=768, use_one_hot=False, embedding_table=Parameter (name=word_embeddings.embedding_table, shape=(30523, 768), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=None>

In [19]:
qFormer2.cls.predictions.bias

Parameter (name=cls.predictions.bias, shape=(30523,), dtype=Float32, requires_grad=True)

In [7]:
# test inputs
import torch
from mindspore import Tensor
query_tokens = torch.load("/mnt/d/QFormer/query_tokens.pth", map_location="cpu")
image_embeds = torch.load("/mnt/d/QFormer/image_embeds.pth", map_location="cpu")
image_atts = torch.load("/mnt/d/QFormer/image_atts.pth", map_location="cpu")


query_tokens = Tensor.from_numpy(query_tokens.detach().numpy())
image_embeds = Tensor.from_numpy(image_embeds.detach().numpy())
image_atts = Tensor.from_numpy(image_atts.detach().numpy())

In [8]:
query_output = qFormer2.bert(
           query_embeds=query_tokens,
           encoder_hidden_states=image_embeds,
           encoder_attention_mask=image_atts,
           use_cache=True,
           return_dict=True,
           )

TypeError: Only support assign to attribute of self, but got attribute of cls.
More details please refer to syntax support at https://www.mindspore.cn

----------------------------------------------------
- The Traceback of Net Construct Code:
----------------------------------------------------

# In file /home/jin/miniconda3/envs/blip2/lib/python3.9/site-packages/mindspore/common/_decorator.py:44
                cls.substitute_name = substitute
                ^

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/pipeline/jit/parse/parse.cc:2718 WriteAssignVars


In [ ]:
embed = qFormer2.bert.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            query_embeds=query_embeds,
            past_key_values_length=past_key_values_length,
        )

In [7]:
qFormer2_params = mindspore_params(qFormer2)

bert.embeddings.position_ids (1, 512)
word_embeddings.embedding_table (30523, 768)
bert.embeddings.position_embeddings.embedding_table (512, 768)
bert.embeddings.LayerNorm.gamma (768,)
bert.embeddings.LayerNorm.beta (768,)
bert.encoder.layer.0.attention.self.query.weight (768, 768)
bert.encoder.layer.0.attention.self.query.bias (768,)
bert.encoder.layer.0.attention.self.key.weight (768, 768)
bert.encoder.layer.0.attention.self.key.bias (768,)
bert.encoder.layer.0.attention.self.value.weight (768, 768)
bert.encoder.layer.0.attention.self.value.bias (768,)
bert.encoder.layer.0.attention.output.dense.weight (768, 768)
bert.encoder.layer.0.attention.output.dense.bias (768,)
bert.encoder.layer.0.attention.output.LayerNorm.gamma (768,)
bert.encoder.layer.0.attention.output.LayerNorm.beta (768,)
bert.encoder.layer.0.crossattention.self.query.weight (768, 768)
bert.encoder.layer.0.crossattention.self.query.bias (768,)
bert.encoder.layer.0.crossattention.self.key.weight (768, 1408)
bert.encoder

cls.predictions.transform.LayerNorm.gamma (768,)
cls.predictions.transform.LayerNorm.beta (768,)
cls.predictions.decoder.weight (30523, 768)
total params:  335


In [13]:
qFormer2.config

PretrainedConfig {
  "_name_or_path": "bert-base-uncased",
  "add_cross_attention": true,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "cross_attention_freq": 2,
  "encoder_width": 1408,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "query_length": 32,
  "transformers_version": "4.25.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30523
}

In [7]:
qFormer2.cls

BertOnlyMLMHead<
  (predictions): BertLMPredictionHead<
    (transform): BertPredictionHeadTransform<
      (dense): Mock_Dense<input_channels=768, output_channels=768, has_bias=True>
      (transform_act_fn): IdentityACT<>
      (LayerNorm): LayerNorm<normalized_shape=[768], begin_norm_axis=-1, begin_params_axis=-1, gammaParameter (name=cls.predictions.transform.LayerNorm.gamma, shape=(768,), dtype=Float32, requires_grad=True), beta=Parameter (name=cls.predictions.transform.LayerNorm.beta, shape=(768,), dtype=Float32, requires_grad=True)>
      >
    (decoder): Mock_Dense<input_channels=768, output_channels=30522>
    >
  >

In [ ]:
# qFormer2_params = mindspore_params(qFormer2)

In [ ]:
from mindspore import numpy as np
from mindspore import Parameter
a = np.randn(4, 2, 4)
b = np.randn(4, 2, 4)

In [ ]:
p = Parameter(a)

In [ ]:
p.assign_value(b)

In [ ]:
qFormer2_params = mindspore_params(qFormer2)

In [ ]:
qFormer2.get_output_embeddings()

In [ ]:
qFormer2.get_output_embeddings().weight.value().ndim

In [ ]:
qFormer2.cls

In [ ]:
from mindspore import ops as P
from mindspore import numpy as np
xx = np.arange(20).reshape(2, 5, 2)

In [ ]:
xxx = P.expand_dims(xx, -1)

In [ ]:
xxx.shape

In [ ]:
xxxx = P.broadcast_to(xxx, (2, -1, -1, -1, -1))

In [ ]:
(xxxx[0] == xxxx[1]).all()

In [ ]:
yy = np.arange(20).reshape(4, 5)

In [ ]:
yyy = P.expand_dims(P.expand_dims(yy, 1), 1)

In [ ]:
(yyy == (yy[:, None, None, :])).all()

In [ ]:
yyy.shape

In [ ]:
yyy.transpose(2, 1, 0, 3)

In [ ]:
yt1 = P.expand_dims(yy, 1)

In [ ]:
yt1.shape

In [ ]:
yt2 = yt1.copy()
yt2 = torch.from_numpy(yt2.asnumpy())

In [ ]:
yt2.transpose(-1, -2).shape

In [ ]:
tmp_shape = [ i for i in range(yt1.dim())]
tmp_shape[-1], tmp_shape[-2] = tmp_shape[-2], tmp_shape[-1]
# tmp_shape = tuple(tmp_shape)
print(tmp_shape)
yt1.transpose(tmp_shape).asnumpy() == yt2.transpose(-1, -2).detach().numpy()

In [ ]:
query_output = qFormer2.bert(
           query_embeds=query_tokens,
           encoder_hidden_states=image_embeds,
           encoder_attention_mask=image_atts,
           use_cache=True,
           return_dict=True,
           )

In [ ]:
query_output.last_hidden_state

### ------------------------------------------------------------ split line ---------------------------------------------------------------

In [ ]:
qFormer2.get_input_embeddings()

In [ ]:
p = qFormer2.get_input_embeddings().embedding_table.value().dim()

In [ ]:
p

In [ ]:
from mindspore import nn
a = nn.Cell()
a.cells()

In [ ]:
def func(f: nn.Cell):
    print(f)
a.apply(func)

In [ ]:
def change_a(value):
    value = None
    return

a = 3

In [ ]:
change_a(a)

In [ ]:
a

In [ ]:
# embed = qFormer2.get_input_embeddings()

In [ ]:
# a, b = embed.embedding_table.shape

In [ ]:
# dense = qFormer2.get_output_embeddings()

In [ ]:
import torch
m = torch.nn.LayerNorm(10)

In [ ]:
m.weight.shape

In [ ]:
m.

In [ ]:
from mindspore import nn
x = nn.LayerNorm([10])

In [ ]:
from mindspore import ops as P

In [ ]:
x.gamma.value()

In [ ]:
x.gamma.init_data()

In [ ]:
x.gamma.set_data(P.zeros_like(x.gamma.value()))

In [ ]:
qFormer2.config.is_encoder_decoder

In [ ]:
# qFormer2.load_state_dict(state_dict)

In [ ]:
len(list(qFormer2.get_parameters()))

In [ ]:
qFormer2.cls.predictions.bias.has_init

In [ ]:
qFormer2_params = mindspore_params(qFormer2)

In [ ]:
print("total params: ", len(state_dict))
for name, tensor in state_dict.items():
    print(name, tensor.shape)

In [ ]:
qFormer2.parameters()

In [ ]:
for i in qFormer2.parameters():
    print(i)

#### convert and save state dict

In [ ]:
def mindspore_params(network):
    ms_params = {}
    param_cnt = 0
    for param in network.get_parameters():
        name = param.name
        value = param.data.value()
        print(name, value.shape)
        ms_params[name] = value
        param_cnt += 1
    print("total params: ", param_cnt)
    return ms_params

In [ ]:
for param in qFormer2.get_parameters():
    print(param.name, param.data.asnumpy())

In [ ]:
ms_params = mindspore_params(qFormer2)

In [13]:
import torch
pt_params = torch.load("/mnt/d/QFormer/self_Qformer_state_dict_init.pth", map_location="cpu")

In [14]:
import mindspore as ms
from mindspore import Tensor
def param_convert(ms_params, pt_params, ckpt_path):
    # 参数名映射字典
    bn_ms2pt = {"gamma": "weight",
                "beta": "bias",
                "embedding_table": "weight"}
    new_params_list = []
    for ms_param in ms_params.keys():
        # 规整化 nn.Embedding和LayerNrom的参数名称
        replace_flag = False
        for replace_word in bn_ms2pt:
            if replace_word in ms_param:
                   replace_flag = True
        if replace_flag:
            ms_param_item = ms_param.split(".")
            pt_param_item = ms_param_item[:-1] + [bn_ms2pt[ms_param_item[-1]]]
            pt_param = ".".join(pt_param_item)
            if pt_param in pt_params and pt_params[pt_param].shape == ms_params[ms_param].shape:
                numpy_value = pt_params[pt_param].detach().numpy()
                new_params_list.append({"name": ms_param, "data": Tensor.from_numpy(numpy_value)})
                print("loading {} from {}".format(ms_param, pt_param))
            else:
                print(ms_param, "not match in pt_params")
        else:
            if ms_param in pt_params and pt_params[ms_param].shape == ms_params[ms_param].shape:
                numpy_value = pt_params[ms_param].detach().numpy()
                new_params_list.append({"name": ms_param, "data": Tensor.from_numpy(numpy_value)})
                print("loading {} from {}".format(ms_param, ms_param))
            else:
                print(ms_param, "not match in pt_params")
    ms.save_checkpoint(new_params_list, ckpt_path)

In [15]:
import mindspore as ms
ckpt_path = "/mnt/d/QFormer/ms_Q_former.ckpt"
param_convert(qFormer2.parameters_dict(), pt_params, ckpt_path)

loading bert.embeddings.position_ids from bert.embeddings.position_ids
word_embeddings.embedding_table not match in pt_params
loading bert.embeddings.position_embeddings.embedding_table from bert.embeddings.position_embeddings.weight
loading bert.embeddings.LayerNorm.gamma from bert.embeddings.LayerNorm.weight
loading bert.embeddings.LayerNorm.beta from bert.embeddings.LayerNorm.bias
loading bert.encoder.layer.0.attention.self.query.weight from bert.encoder.layer.0.attention.self.query.weight
loading bert.encoder.layer.0.attention.self.query.bias from bert.encoder.layer.0.attention.self.query.bias
loading bert.encoder.layer.0.attention.self.key.weight from bert.encoder.layer.0.attention.self.key.weight
loading bert.encoder.layer.0.attention.self.key.bias from bert.encoder.layer.0.attention.self.key.bias
loading bert.encoder.layer.0.attention.self.value.weight from bert.encoder.layer.0.attention.self.value.weight
loading bert.encoder.layer.0.attention.self.value.bias from bert.encoder.l

In [16]:
# qFormer2.parameters_dict()

In [19]:
import mindspore as ms
ckpt_path = "/mnt/d/QFormer/ms_Q_former.ckpt"
# param_convert(qFormer2.parameters_dict(), pt_params, ckpt_path)
ms_param_dict = ms.load_checkpoint(ckpt_path)
param_not_load = ms.load_param_into_net(qFormer2, ms_param_dict)

[WARNING] ME(16810:140443486904704,MainProcess):2023-06-13-10:39:10.238.71 [mindspore/train/serialization.py:163] The type of bert.embeddings.position_ids:Int64 in 'parameter_dict' is different from the type of it in 'net':Int32, then the type convert from Int64 to Int32 in the network.


In [20]:
param_not_load

[]

#### forward compute validation

In [21]:
# test inputs
import torch
from mindspore import Tensor
query_tokens = torch.load("/mnt/d/QFormer/query_tokens.pth", map_location="cpu")
image_embeds = torch.load("/mnt/d/QFormer/image_embeds.pth", map_location="cpu")
image_atts = torch.load("/mnt/d/QFormer/image_atts.pth", map_location="cpu")


query_tokens = Tensor.from_numpy(query_tokens.detach().numpy())
image_embeds = Tensor.from_numpy(image_embeds.detach().numpy())
image_atts = Tensor.from_numpy(image_atts.detach().numpy())

In [22]:
query_output2 = qFormer2.bert(
           query_embeds=query_tokens,
           encoder_hidden_states=image_embeds,
           encoder_attention_mask=image_atts,
           use_cache=True,
           return_dict=True,
           )

In [24]:
hs2 = query_output2.last_hidden_state

In [27]:
hs2

Tensor(shape=[100, 32, 768], dtype=Float32, value=
[[[ 6.36275828e-01, -1.23322546e+00, -2.64354914e-01 ...  3.03988844e-01, -1.24536790e-01,  1.63442934e+00],
  [ 6.29782557e-01, -1.23557854e+00, -2.65074492e-01 ...  3.06480467e-01, -1.22025825e-01,  1.63047945e+00],
  [ 6.36725545e-01, -1.23086798e+00, -2.63852268e-01 ...  3.02246600e-01, -1.23068646e-01,  1.63454282e+00],
  ...
  [ 6.39091134e-01, -1.22862041e+00, -2.61913985e-01 ...  2.99779266e-01, -1.24267742e-01,  1.63753891e+00],
  [ 6.36037171e-01, -1.23141396e+00, -2.63945043e-01 ...  3.02295268e-01, -1.23219654e-01,  1.63482201e+00],
  [ 6.40985966e-01, -1.22782278e+00, -2.62156427e-01 ...  2.97930896e-01, -1.24372512e-01,  1.63825858e+00]],
 [[ 7.07381606e-01, -1.16373205e+00, -2.78238177e-01 ...  2.48608321e-01, -2.10301340e-01,  1.58638465e+00],
  [ 7.00246751e-01, -1.16481912e+00, -2.79877335e-01 ...  2.50751168e-01, -2.08516896e-01,  1.58291590e+00],
  [ 7.07308173e-01, -1.16301501e+00, -2.77706742e-01 ...  2.47928858e-

In [31]:
qFormer2_dict = qFormer2.parameters_dict()
for param in qFormer2.get_parameters():
    if "_query" in param.name:
        key_orig = param.name.replace("_query", "")
        param.data.assign_value(qFormer2_dict[key_orig].data.value())

In [ ]:
query_output3 = qFormer2.bert(
           query_embeds=query_tokens,
           encoder_hidden_states=image_embeds,
           encoder_attention_mask=image_atts,
           use_cache=True,
           return_dict=True,
           )

NameError: name 'qFormer2' is not defined

> /tmp/ipykernel_17015/3432003637.py(1)<module>()
----> 1 query_output3 = qFormer2.bert(
      2            query_embeds=query_tokens,
      3            encoder_hidden_states=image_embeds,
      4            encoder_attention_mask=image_atts,
      5            use_cache=True,



In [60]:
hs3 = query_output3.last_hidden_state

In [61]:
hs3

Tensor(shape=[100, 32, 768], dtype=Float32, value=
[[[-1.66873068e-01,  2.97427177e-01, -2.62021162e-02 ...  3.27335268e-01,  2.37790465e-01,  3.30682784e-01],
  [-1.92414388e-01,  3.16561610e-01, -2.34991126e-02 ...  3.23027343e-01,  2.48318821e-01,  2.72178739e-01],
  [-1.43714860e-01,  2.96599984e-01, -3.72814797e-02 ...  3.19388598e-01,  2.52460420e-01,  3.09142053e-01],
  ...
  [-1.56098589e-01,  2.99248397e-01, -2.65768729e-02 ...  3.21254343e-01,  2.53752828e-01,  3.21896255e-01],
  [-1.66216165e-01,  3.07389528e-01, -3.43378522e-02 ...  3.24267775e-01,  2.46809602e-01,  3.08967233e-01],
  [-1.83831573e-01,  3.27633590e-01, -5.46987914e-02 ...  3.09363157e-01,  2.32047603e-01,  3.47589821e-01]],
 [[-1.41434312e-01,  4.00380135e-01, -1.92433968e-03 ...  4.00864512e-01,  1.98261693e-01,  3.16282243e-01],
  [-1.73103839e-01,  4.23748493e-01, -1.23587251e-03 ...  3.97895575e-01,  2.10109860e-01,  2.52999455e-01],
  [-1.12684198e-01,  4.02132392e-01, -1.48271471e-02 ...  3.98332119e-